In [1]:
import numpy as np 
import pandas as pd

from PIL import Image

import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F

import albumentations
import torchvision
from torchvision import transforms

import timm
import tqdm

from src.losses import ArcMarginProduct
from src.utils.metrics import accuracy
from src.utils.data_loaders import ImagesDS
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
path_data = '/home/marchevskiy/Data/other'
device = 'cuda'
batch_size = 16

In [4]:
transforms_train = albumentations.Compose([
    # albumentations.RandomCrop(height=334, width=334, p=1),
    albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=0.3),
    albumentations.RandomRotate90(p=0.3),
    albumentations.Flip(p=0.3),
    # albumentations.ElasticTransform(alpha=0.5, sigma=25, alpha_affine=25, p=0.3),
    # albumentations.RandomBrightness(limit=0.3, p=0.3),
    # albumentations.RandomContrast(limit=0.15, p=0.3),
    albumentations.Normalize(
     mean=[0.01639522, 0.06704897, 0.0366343 , 0.03555733, 0.02223296, 0.03011626],
     std=[0.01954379, 0.05231356, 0.01980171, 0.02657354, 0.01802711, 0.01548923])
])

transforms_test = albumentations.Compose([
    # albumentations.RandomCrop(height=334, width=334, p=1),
    albumentations.Normalize(
     mean=[0.01639522, 0.06704897, 0.0366343 , 0.03555733, 0.02223296, 0.03011626],
     std=[0.01954379, 0.05231356, 0.01980171, 0.02657354, 0.01802711, 0.01548923])
])

In [5]:
ds = ImagesDS(pd.read_csv(path_data+'/train.csv'), path_data, None, transforms_train, mode='train')
loader = D.DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=8)

In [6]:
class ResNext50(nn.Module):
    def __init__(self, num_classes=1108):
        super().__init__()
        preloaded = torchvision.models.resnext50_32x4d(pretrained=True)
        preloaded.conv1 = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        preloaded.fc = nn.Linear(2048, num_classes, bias=True)
        self.features = nn.Sequential(*list(preloaded.children())[:-2])
        self.dp1 = nn.Dropout(0.6)
        self.classifier = nn.Linear(2048, num_classes, bias=True)
        del preloaded
        
    def forward(self, x):
        features = self.features(x)
        out = F.adaptive_avg_pool2d(features, (1, 1)).view(features.size(0), -1)
        out = self.dp1(out)
        out = self.classifier(out)
        return out

In [7]:
class ResNet50am(nn.Module):
    def __init__(self, num_classes=1108):
        super().__init__()
        preloaded = ResNext50()
        # preloaded.load_state_dict(torch.load('../../data/resnext50_v1(d05(384)).pth'))
        self.features = nn.Sequential(*list(preloaded.children())[:-2])
        self.fc1 = nn.Linear(4096, 512)
        self.nl = AngleSimpleLinear(512, num_classes)
        del preloaded
        
    def forward(self, x):
        x = nn.BatchNorm2d(6)(x)
        x = self.features(x)
        x = torch.cat((nn.AdaptiveAvgPool2d(1)(x), nn.AdaptiveMaxPool2d(1)(x)), dim=1)
        x = x.view(x.size(0), -1)
        x = F.dropout(x, p=0.25)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.nl(x)
        return x

In [10]:
model = ResNet50am()
model.to(device);

In [12]:
criterion = AMSoftmaxLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=30e-5)

epochs = 1
tlen = len(loader)
for epoch in range(epochs):
    model.train()
    tloss = 0
    acc = np.zeros(1)
    for data in tqdm.tqdm_notebook(loader):
        data_input, label = data
        data_input = data_input.to(device)
        label = label.to(device).long()
        output = model(data_input)
        loss = criterion(output, label)
        tloss += loss.item() 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc += accuracy(output.detach().cpu(), label.cpu())
    print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))


Epoch 1 -> Train Loss: 19.6796, ACC: 65.53%


In [13]:
for epoch in range(epochs):
    model.train()
    tloss = 0
    acc = np.zeros(1)
    for data in tqdm.tqdm_notebook(loader):
        data_input, label = data
        data_input = data_input.to(device)
        label = label.to(device).long()
        output = model(data_input)
        loss = criterion(output, label)
        tloss += loss.item() 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc += accuracy(output.detach().cpu(), label.cpu())
    print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))


Epoch 1 -> Train Loss: 19.0773, ACC: 71.13%


In [22]:
torch.save(model.state_dict(), '../../data/resnext50_am.pth')

In [11]:
model.load_state_dict(torch.load('../../data/resnext50_am.pth'));

In [9]:
train_df = pd.read_csv(path_data+'/train.csv')
train_df_HEPG2 = train_df[train_df['experiment'].isin(['HEPG2-01', 'HEPG2-02', 'HEPG2-03', 'HEPG2-04', 'HEPG2-05',
                                                       'HEPG2-06', 'HEPG2-07'])]

In [10]:
n_folds = 4
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=0)
fold = folds.split(train_df_HEPG2, train_df_HEPG2['sirna'])

In [11]:
model = ResNet50am()
model.to(device);

In [13]:
for fold_, (train_index, val_index) in enumerate(fold):
    best_val_acc = 0
    ds_train = ImagesDS(train_df_HEPG2.iloc[train_index,:], path_data, None, transforms_train, mode='train')
    loader_train = D.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=4)
    ds_valid = ImagesDS(train_df_HEPG2.iloc[val_index,:], path_data, None, transforms_train, mode='train')
    loader_valid = D.DataLoader(ds_valid, batch_size=batch_size, shuffle=False, num_workers=4)
    
    model.load_state_dict(torch.load('../../data/resnext50_am.pth'))

    criterion = AMSoftmaxLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=30e-5)

    epochs = 2
    for epoch in range(epochs):
        model.train()
        tloss = 0
        acc = np.zeros(1)
        tlen = len(loader_train)
        for data in tqdm.tqdm_notebook(loader_train):
            data_input, label = data
            data_input = data_input.to(device)
            label = label.to(device).long()
            output = model(data_input)
            loss = criterion(output, label)
            tloss += loss.item() 
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            acc += accuracy(output.detach().cpu(), label.cpu())
            del loss, output, label, data_input
        print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))
        if epoch > 5:
            optimizer.param_groups[0]['lr'] = 15e-5
        if epoch > 10:
            optimizer.param_groups[0]['lr'] = 7.5e-5
        if epoch > 15:
            optimizer.param_groups[0]['lr'] = 3e-5
        
        acc = np.zeros(1)
        tlen = len(loader_valid)
        model.eval()
        for x, y in loader_valid:
            x = x.to(device)
            output = model(x)
            acc += accuracy(output.cpu(), y)
            del output, x, y
        print('Epoch {} -> Valid ACC: {:.2f}%'.format(epoch+1, acc[0]/tlen))
        
        if (acc[0]/tlen) > best_val_acc:
            torch.save(model.state_dict(), '../../data/resnext50_am_HEPG2_fold{}.pth'.format(fold_))
            best_val_acc = acc[0]/tlen


Epoch 1 -> Train Loss: 20.1989, ACC: 58.21%
Epoch 1 -> Valid ACC: 70.91%



Epoch 2 -> Train Loss: 19.2090, ACC: 72.34%
Epoch 2 -> Valid ACC: 75.48%



Epoch 1 -> Train Loss: 20.2018, ACC: 58.71%
Epoch 1 -> Valid ACC: 73.19%



Epoch 2 -> Train Loss: 19.2236, ACC: 72.61%
Epoch 2 -> Valid ACC: 76.31%



Epoch 1 -> Train Loss: 20.1225, ACC: 59.21%
Epoch 1 -> Valid ACC: 74.01%



Epoch 2 -> Train Loss: 19.1482, ACC: 72.99%
Epoch 2 -> Valid ACC: 77.83%


In [14]:
train_df = pd.read_csv(path_data+'/train.csv')
train_df_U2OS = train_df[train_df['experiment'].isin(['U2OS-01', 'U2OS-02', 'U2OS-03'])]

In [15]:
n_folds = 4
folds = KFold(n_splits=n_folds, shuffle=True, random_state=0)
fold = folds.split(train_df_U2OS, train_df_U2OS['sirna'])

In [19]:
for fold_, (train_index, val_index) in enumerate(fold):
    best_val_acc = 0
    ds_train = ImagesDS(train_df_U2OS.iloc[train_index,:], path_data, None, transforms_train, mode='train')
    loader_train = D.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=4)
    ds_valid = ImagesDS(train_df_U2OS.iloc[val_index,:], path_data, None, transforms_train, mode='train')
    loader_valid = D.DataLoader(ds_valid, batch_size=batch_size, shuffle=False, num_workers=4)
    
    model.load_state_dict(torch.load('../../data/resnext50_am.pth'))

    criterion = AMSoftmaxLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=30e-5)

    epochs = 2
    tlen = len(loader_train)
    for epoch in range(epochs):
        model.train()
        tloss = 0
        acc = np.zeros(1)
        for data in tqdm.tqdm_notebook(loader_train):
            data_input, label = data
            data_input = data_input.to(device)
            label = label.to(device).long()
            output = model(data_input)
            loss = criterion(output, label)
            tloss += loss.item() 
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            acc += accuracy(output.detach().cpu(), label.cpu())
            del loss, output, label, data_input
        print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))
        if epoch > 5:
            optimizer.param_groups[0]['lr'] = 15e-5
        if epoch > 10:
            optimizer.param_groups[0]['lr'] = 7.5e-5
        if epoch > 15:
            optimizer.param_groups[0]['lr'] = 3e-5
        
        acc = np.zeros(1)
        tlen = len(loader_valid)
        model.eval()
        for x, y in loader_valid:
            x = x.to(device)
            output = model(x)
            acc += accuracy(output.cpu(), y)
            del output, x, y
        print('Epoch {} -> Valid ACC: {:.2f}%'.format(epoch+1, acc[0]/tlen))
        
        if (acc[0]/tlen) > best_val_acc:
            torch.save(model.state_dict(), '../../data/resnext50_am_U2OS_fold{}.pth'.format(fold_))
            best_val_acc = acc[0]/tlen


Epoch 1 -> Train Loss: 21.4369, ACC: 36.33%
Epoch 1 -> Valid ACC: 41.71%



Epoch 2 -> Train Loss: 60.1029, ACC: 189.41%
Epoch 2 -> Valid ACC: 47.04%



Epoch 1 -> Train Loss: 21.4393, ACC: 36.45%
Epoch 1 -> Valid ACC: 43.42%



Epoch 2 -> Train Loss: 60.0716, ACC: 192.33%
Epoch 2 -> Valid ACC: 48.97%


In [20]:
train_df = pd.read_csv(path_data+'/train.csv')
train_df_HUVEC = train_df[train_df['experiment'].isin(['HUVEC-01', 'HUVEC-02', 'HUVEC-03', 'HUVEC-04',
                                                     'HUVEC-05', 'HUVEC-06', 'HUVEC-07', 'HUVEC-08',
                                                     'HUVEC-09', 'HUVEC-10', 'HUVEC-11', 'HUVEC-12',
                                                     'HUVEC-13', 'HUVEC-14', 'HUVEC-15', 'HUVEC-16'])]

In [21]:
n_folds = 4
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=0)
fold = folds.split(train_df_HUVEC, train_df_HUVEC['sirna'])

In [22]:
for fold_, (train_index, val_index) in enumerate(fold):
    best_val_acc = 0
    ds_train = ImagesDS(train_df_HUVEC.iloc[train_index,:], path_data, None, transforms_train, mode='train')
    loader_train = D.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=4)
    ds_valid = ImagesDS(train_df_HUVEC.iloc[val_index,:], path_data, None, transforms_train, mode='train')
    loader_valid = D.DataLoader(ds_valid, batch_size=batch_size, shuffle=False, num_workers=4)
    
    model.load_state_dict(torch.load('../../data/resnext50_am.pth'))

    criterion = AMSoftmaxLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=30e-5)

    epochs = 20
    for epoch in range(epochs):
        model.train()
        tloss = 0
        acc = np.zeros(1)
        tlen = len(loader_train)
        for data in tqdm.tqdm_notebook(loader_train):
            data_input, label = data
            data_input = data_input.to(device)
            label = label.to(device).long()
            output = model(data_input)
            loss = criterion(output, label)
            tloss += loss.item() 
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            acc += accuracy(output.detach().cpu(), label.cpu())
            del loss, output, label, data_input
        print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))
        if epoch > 5:
            optimizer.param_groups[0]['lr'] = 15e-5
        if epoch > 10:
            optimizer.param_groups[0]['lr'] = 7.5e-5
        if epoch > 15:
            optimizer.param_groups[0]['lr'] = 3e-5
        
        acc = np.zeros(1)
        tlen = len(loader_valid)
        model.eval()
        for x, y in loader_valid:
            x = x.to(device)
            output = model(x)
            acc += accuracy(output.cpu(), y)
            del output, x, y
        print('Epoch {} -> Valid ACC: {:.2f}%'.format(epoch+1, acc[0]/tlen))
        
        if (acc[0]/tlen) > best_val_acc:
            torch.save(model.state_dict(), '../../data/resnext50_am_HUVEC_fold{}.pth'.format(fold_))
            best_val_acc = acc[0]/tlen


Epoch 1 -> Train Loss: 17.1295, ACC: 87.45%
Epoch 1 -> Valid ACC: 92.11%


KeyboardInterrupt: 

In [ ]:
train_df = pd.read_csv(path_data+'/train.csv')
train_df_RPE = train_df[train_df['experiment'].isin(['RPE-01', 'RPE-02', 'RPE-03', 'RPE-04',
                                                     'RPE-05', 'RPE-06', 'RPE-07'])]

In [ ]:
n_folds = 4
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=0)
fold = folds.split(train_df_RPE, train_df_RPE['sirna'])

In [ ]:
for fold_, (train_index, val_index) in enumerate(fold):
    best_val_acc = 0
    ds_train = ImagesDS(train_df_RPE.iloc[train_index,:], path_data, None, transforms_train, mode='train')
    loader_train = D.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=4)
    ds_valid = ImagesDS(train_df_RPE.iloc[val_index,:], path_data, None, transforms_train, mode='train')
    loader_valid = D.DataLoader(ds_valid, batch_size=batch_size, shuffle=False, num_workers=4)
    
    model.load_state_dict(torch.load('../../data/resnext50_am.pth'))

    criterion = AMSoftmaxLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=30e-5)

    epochs = 20
    for epoch in range(epochs):
        model.train()
        tloss = 0
        acc = np.zeros(1)
        tlen = len(loader_train)
        for data in tqdm.tqdm_notebook(loader_train):
            data_input, label = data
            data_input = data_input.to(device)
            label = label.to(device).long()
            output = model(data_input)
            loss = criterion(output, label)
            tloss += loss.item() 
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            acc += accuracy(output.detach().cpu(), label.cpu())
            del loss, output, label, data_input
        print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))
        if epoch > 5:
            optimizer.param_groups[0]['lr'] = 15e-5
        if epoch > 10:
            optimizer.param_groups[0]['lr'] = 7.5e-5
        if epoch > 15:
            optimizer.param_groups[0]['lr'] = 3e-5
        
        acc = np.zeros(1)
        tlen = len(loader_valid)
        model.eval()
        for x, y in loader_valid:
            x = x.to(device)
            output = model(x)
            acc += accuracy(output.cpu(), y)
            del output, x, y
        print('Epoch {} -> Valid ACC: {:.2f}%'.format(epoch+1, acc[0]/tlen))
        
        if (acc[0]/tlen) > best_val_acc:
            torch.save(model.state_dict(), '../../data/resnext50_am_RPE_fold{}.pth'.format(fold_))
            best_val_acc = acc[0]/tlen

In [18]:
tds = ImagesDS(pd.read_csv(path_data+'/test.csv'), path_data, 2, transforms_test, mode='test')
tloader = D.DataLoader(tds, batch_size=batch_size, shuffle=False, num_workers=8)

In [19]:
model.eval()

@torch.no_grad()
def prediction(model, loader):
    outputs = []
    preds = np.empty(0)
    for x, _ in loader: 
        x = x.to(device)
        output = model(x)
        outputs.append(output)
        idx = output.max(dim=-1)[1].cpu().numpy()
        preds = np.append(preds, idx, axis=0)
    return preds, outputs

preds1, outputs2 = prediction(model, tloader)

In [20]:
preds = ((torch.cat(outputs1)+torch.cat(outputs2)) / 2.).max(dim=-1)[1].cpu().numpy()

In [21]:
submission = pd.read_csv(path_data + '/test.csv')
submission['sirna'] = preds.astype(int)
submission.to_csv('submission.csv', index=False, columns=['id_code','sirna'])

In [8]:
import math
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred    = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))

    return res


class AngleSimpleLinear(nn.Module):
    """Computes cos of angles between input vectors and weights vectors"""
    def __init__(self, in_features, out_features):
        super(AngleSimpleLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(in_features, out_features))
        self.weight.data.uniform_(-1, 1).renorm_(2, 1, 1e-5).mul_(1e5)

    def forward(self, x):
        cos_theta = F.normalize(x, dim=1).mm(F.normalize(self.weight, dim=0))
        return cos_theta.clamp(-1, 1)


def focal_loss(input_values, gamma):
    """Computes the focal loss"""
    p = torch.exp(-input_values)
    loss = (1 - p) ** gamma * input_values
    return loss.mean()


class AMSoftmaxLoss(nn.Module):
    """Computes the AM-Softmax loss with cos or arc margin"""
    margin_types = ['cos', 'arc']

    def __init__(self, margin_type='cos', gamma=2., m=0.5, s=30, t=1.2):
        super(AMSoftmaxLoss, self).__init__()
        assert margin_type in AMSoftmaxLoss.margin_types
        self.margin_type = margin_type
        assert gamma >= 0
        self.gamma = gamma
        assert m > 0
        self.m = m
        assert s > 0
        self.s = s
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        assert t >= 1
        self.t = t

    def forward(self, cos_theta, target):
        if self.margin_type == 'cos':
            phi_theta = cos_theta - self.m
        else:
            sine = torch.sqrt(1.0 - torch.pow(cos_theta, 2))
            phi_theta = cos_theta * self.cos_m - sine * self.sin_m #cos(theta+m)
            phi_theta = torch.where(cos_theta > self.th, phi_theta, cos_theta - self.sin_m * self.m)

        index = torch.zeros_like(cos_theta, dtype=torch.uint8)
        index.scatter_(1, target.data.view(-1, 1), 1)
        output = torch.where(index, phi_theta, cos_theta)

        if self.gamma == 0 and self.t == 1.:
            return F.cross_entropy(self.s*output, target)

        if self.t > 1:
            h_theta = self.t - 1 + self.t*cos_theta
            support_vecs_mask = (1 - index) * \
                torch.lt(torch.masked_select(phi_theta, index).view(-1, 1).repeat(1, h_theta.shape[1]) - cos_theta, 0)
            output = torch.where(support_vecs_mask, h_theta, output)
            return F.cross_entropy(self.s*output, target)

        return focal_loss(F.cross_entropy(self.s*output, target, reduction='none'), self.gamma)